![Alt Text](https://raw.githubusercontent.com/msfasha/307304-Data-Mining/main/20242/images/header.png)

<div style="display: flex; justify-content: flex-start; align-items: center;">
   <a href="https://colab.research.google.com/github/msfasha/307307-BI-Methods/blob/main/20242-NLP-LLM/Part%203%20-%20Introduction%20to%20DL%20and%20LLMs/1-Introduction%20to%20DL%20and%20LLMs.ipynb" target="_parent"><img 
   src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</div>

### Hugging Face Pipelines Tutorial
Hugging Face Pipelines provide a simple, high-level interface for using pre-trained models for various NLP, computer vision, and audio tasks. They abstract away much of the complexity involved in preprocessing, model inference, and postprocessing.

#### Installation
First, install the transformers library:

In [1]:
! pip install transformers torch

# For additional features:
! pip install transformers[torch] datasets evaluate

   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   ------ --------------------------------- 3.9/25.8 MB 18.0 MB/s eta 0:00:02
   ------------- -------------------------- 8.9/25.8 MB 21.3 MB/s eta 0:00:01
   --------------------- ------------------ 13.9/25.8 MB 21.8 MB/s eta 0:00:01
   --------------------------- ------------ 17.8/25.8 MB 20.8 MB/s eta 0:00:01
   --------------------------------- ------ 21.5/25.8 MB 20.3 MB/s eta 0:00:01
   ---------------------------------------  25.4/25.8 MB 20.1 MB/s eta 0:00:01
   ---------------------------------------- 25.8/25.8 MB 19.7 MB/s eta 0:00:00

   -- -------------------------------------  1/16 [pyarrow]
   -- -------------------------------------  1/16 [pyarrow]
   -- -------------------------------------  1/16 [pyarrow]
   -- -------------------------------------  1/16 [pyarrow]
   -- -------------------------------------  1/16 [pyarrow]
   -- -------------------------------------  1/16 [pyarrow]
   -- ------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.7.0+cpu requires torch==2.7.0, but you have torch 2.1.2+cpu which is incompatible.


## Advanced Pipeline Usage

### Specifying Custom Models

In [ ]:
# Use a specific model
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

# Test with social media text
tweet = "Just had the best coffee ever! ☕ #perfect"
result = classifier(tweet)
print(f"Tweet: {tweet}")
print(f"Sentiment: {result[0]['label']}, Score: {result[0]['score']:.4f}")

### Batch Processing

In [ ]:
# Efficient batch processing
texts = [
    "I love this product!",
    "This is terrible.",
    "It's okay.",
    "Absolutely amazing!",
    "Could be better."
]

# Process in batches for efficiency
classifier = pipeline("sentiment-analysis")
results = classifier(texts, batch_size=2)

for text, result in zip(texts, results):
    print(f"{text} -> {result['label']} ({result['score']:.3f})")

### Device Configuration

In [ ]:
import torch

# Check if CUDA is available
device = 0 if torch.cuda.is_available() else -1

# Create pipeline with specific device
classifier = pipeline(
    "sentiment-analysis",
    device=device  # 0 for GPU, -1 for CPU
)

# The pipeline will now run on GPU if available
result = classifier("This will run faster on GPU!")
print(result)

## Computer Vision Pipelines

### Image Classification

In [ ]:
from PIL import Image
import requests

# Image classification pipeline
classifier = pipeline("image-classification")

# Load image from URL
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
image = Image.open(requests.get(url, stream=True).raw)

# Classify image
results = classifier(image)
print("Top predictions:")
for result in results[:3]:
    print(f"Label: {result['label']}, Score: {result['score']:.4f}")

### Object Detection

In [ ]:
# Object detection pipeline
detector = pipeline("object-detection")

# Detect objects in image
results = detector(image)

print(f"Found {len(results)} objects:")
for result in results:
    box = result['box']
    print(f"Label: {result['label']}")
    print(f"Score: {result['score']:.4f}")
    print(f"Box: {box}\n")

## Audio Pipelines

### Automatic Speech Recognition

In [ ]:
# Note: You'll need to install additional dependencies
# pip install librosa soundfile

# ASR pipeline
transcriber = pipeline("automatic-speech-recognition")

# Transcribe audio file
# audio_file = "path/to/your/audio.wav"
# result = transcriber(audio_file)
# print(f"Transcription: {result['text']}")

## Custom Preprocessing and Postprocessing

In [ ]:
# Create a custom pipeline with preprocessing
def preprocess_text(text):
    # Remove extra whitespace and convert to lowercase
    return text.strip().lower()

def postprocess_result(result):
    # Add custom formatting
    label = result['label']
    score = result['score']
    
    if score > 0.9:
        confidence = "Very confident"
    elif score > 0.7:
        confidence = "Confident"
    else:
        confidence = "Less confident"
    
    return {
        'prediction': label,
        'confidence_level': confidence,
        'raw_score': score
    }

# Usage example
classifier = pipeline("sentiment-analysis")

text = "   THIS IS AMAZING!!!   "
preprocessed = preprocess_text(text)
result = classifier(preprocessed)[0]
final_result = postprocess_result(result)

print(f"Original: {text}")
print(f"Preprocessed: {preprocessed}")
print(f"Final result: {final_result}")

## Pipeline with Return Tensors

In [ ]:
# Get raw model outputs
classifier = pipeline("sentiment-analysis", return_all_scores=True)

text = "I'm not sure how I feel about this."
results = classifier(text)

print("All scores:")
for result in results[0]:  # Pipeline returns list of lists
    print(f"{result['label']}: {result['score']:.4f}")

## Error Handling and Best Practices

In [ ]:
from transformers import pipeline
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

try:
    # Create pipeline with error handling
    classifier = pipeline("sentiment-analysis")
    
    # Handle various input types
    inputs = [
        "Normal text",
        "",  # Empty string
        "A" * 1000,  # Very long text
        None,  # None value
    ]
    
    for inp in inputs:
        try:
            if inp is None:
                print(f"Input: {inp} -> Skipped (None value)")
                continue
                
            if len(inp) == 0:
                print(f"Input: '{inp}' -> Skipped (empty string)")
                continue
                
            result = classifier(inp)
            print(f"Input: {inp[:50]}... -> {result[0]['label']}")
            
        except Exception as e:
            print(f"Error processing '{inp[:50]}...': {str(e)}")
            
except Exception as e:
    print(f"Error creating pipeline: {str(e)}")

## Performance Tips

In [ ]:
# 1. Reuse pipelines instead of creating new ones
classifier = pipeline("sentiment-analysis")

# 2. Use batch processing for multiple inputs
texts = ["Text 1", "Text 2", "Text 3"]
results = classifier(texts)  # More efficient than individual calls

# 3. Adjust max_length for your use case
classifier = pipeline("sentiment-analysis", max_length=128, truncation=True)

# 4. Use appropriate model size for your needs
# smaller models: distilbert-base-uncased-finetuned-sst-2-english
# larger models: roberta-large-mnli

---

## Build a Sentiment Analyzer in Hugging Face Spaces

### Introduction

This guide walks you through building and deploying a sentiment analysis web app using Hugging Face Spaces. The app uses a pre-trained model and requires no local development environment.

### What You Need

* Hugging Face account
* No local setup required
* Basic understanding of Python and Transformers

### Step 1: Create a New Space

1. Go to [https://huggingface.co/spaces](https://huggingface.co/spaces)
2. Click **“Create new Space”**
3. Fill in:

   * **Space name**
   * **SDK:** Gradio
   * **Visibility:** Public or Private
4. Click **“Create Space”**

### Step 2: Add Application Files

Once your space is created, you will see a file manager interface.

#### File: `app.py`

```python
from transformers import pipeline
import gradio as gr

classifier = pipeline("sentiment-analysis")

def analyze_sentiment(text):
    result = classifier(text)[0]
    return f"{result['label']} ({round(result['score'] * 100, 2)}%)"

interface = gr.Interface(
    fn=analyze_sentiment,
    inputs="text",
    outputs="text",
    title="Sentiment Analyzer"
)

interface.launch()
```

#### File: `requirements.txt`

```
transformers
gradio
```

Upload both files to your Hugging Face Space.

### Step 3: Deploy

Once the files are uploaded:

* Hugging Face will automatically install dependencies
* The app will be built and hosted automatically
* A public URL will be provided to access and share your app

### Enhancements

* Try different models (e.g., multilingual sentiment models)
* Add visual feedback like confidence scores as bars or progress indicators
* Include explanation metadata using libraries like `transformers-interpret`
